<a href="https://colab.research.google.com/github/klaxonlet/CE888_2020/blob/master/project1_main_dsdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
#get_ipython().magic('matplotlib inline')


In [0]:
%cd /content/
!mkdir data_pollution
!mkdir data_housing
%cd data_pollution

In [0]:
os.environ['KAGGLE_USERNAME'] = "klaxonlet" 
os.environ['KAGGLE_KEY'] = "6a538fd5e6037f2d581d4dd78fe1aeba" 
!kaggle datasets download -d bappekim/air-pollution-in-seoul

In [0]:
%cd /content/data_housing/
os.environ['KAGGLE_USERNAME'] = "klaxonlet" 
os.environ['KAGGLE_KEY'] = "6a538fd5e6037f2d581d4dd78fe1aeba" 
!kaggle competitions download -c sberbank-russian-housing-market

In [0]:
!unzip test.csv.zip
!unzip train.csv.zip

In [0]:
%cd /content/data_pollution/
!unzip air-pollution-in-seoul.zip

Checking for covariate shift in air pollution dataset

In [0]:
%cd AirPollutionSeoul
data_pollution = pd.read_csv('Measurement_summary.csv') # loading our Air Pollution dataset.
data_pollution.isna().sum() # Checking for missing values in Air Pollution data

In [0]:
data_pollution_target = data_pollution['CO']  # Saving target variable for test-train split
data_pollution = data_pollution.drop('CO',axis=1) # Dropping target variable 

In [0]:
# Splitting data into test-train split
train_pollution, test_pollution, y_train_pollution, y_test_pollution = train_test_split(data_pollution, data_pollution_target, test_size=0.33, random_state=42)

In [0]:
#Label encoding
label = LabelEncoder()
for i in train_pollution.columns:
    if (train_pollution[i].dtype == 'object'):
      train_pollution[i] = label.fit_transform(train_pollution[i].astype('str'))
      train_pollution[i] = train_pollution[i].astype('object')

for i in test_pollution.columns:
    if (test_pollution[i].dtype == 'object'):
      test_pollution[i] = label.fit_transform(test_pollution[i].astype('str'))
      test_pollution[i] = test_pollution[i].astype('object')

In [0]:
# Creating new variable 'origin'
train_pollution['origin'] = 0
test_pollution['origin'] = 1

In [0]:
test_pollution.describe()

In [0]:
train_pollution.describe()

In [0]:
# Creating random samples from train-test data and combining them.
sample_train_pollution = train_pollution.sample(213679, random_state=12)
sample_test_pollution = test_pollution.sample(212000, random_state=11)  

sample_combined_pollution = sample_train_pollution.append(sample_test_pollution)


In [0]:
sample_y_combined_pollution = sample_combined_pollution['origin'] # saving our 'origin' target variable.
sample_combined_pollution = sample_combined_pollution.drop('origin',axis=1) # dropping target variable from data for modelling.

In [124]:
# Creating model
model_sample_pollution = RandomForestClassifier(n_estimators = 50, max_depth = 5,min_samples_leaf = 5)
drop_list_pollution = []
for i in sample_combined_pollution.columns:
  score = cross_val_score(model_sample_pollution,pd.DataFrame(sample_combined_pollution[i]),sample_y_combined_pollution,cv=2,scoring='roc_auc')
  if (np.mean(score) > 0.8):
    drop_list.append(i)
  print(i,np.mean(score))

Measurement date 0.5005021416422071
Station code 0.4986458731725534
Address 0.4982339374933911
Latitude 0.49864596096536484
Longitude 0.49839290730152863
SO2 0.4989950561257523
NO2 0.5004065404333504
O3 0.4998280274102176
PM10 0.4994099843786465
PM2.5 0.49988216917456835


Checking for covariate shift in our housing market dataset

In [0]:
%cd /content/data_housing/

In [0]:
train_housing = pd.read_csv('train.csv') # loading our housing dataset.
test_housing = pd.read_csv('test.csv') # loading our housing dataset.

train_housing.isna().sum() # Checking for missing values in housing data

In [0]:
# Imputing missing values
for i in train_housing.columns:
    if train_housing[i].dtype == 'object':
      train_housing[i] = train_housing[i].fillna(train_housing[i].mode().iloc[0])
    if (train_housing[i].dtype == 'int' or train_housing[i].dtype == 'float'):
      train_housing[i] = train_housing[i].fillna(np.mean(train_housing[i]))


for i in test_housing.columns:
    if test_housing[i].dtype == 'object':
      test_housing[i] = test_housing[i].fillna(test_housing[i].mode().iloc[0])
    if (test_housing[i].dtype == 'int' or test_housing[i].dtype == 'float'):
      test_housing[i] = test_housing[i].fillna(np.mean(test_housing[i]))

In [0]:
#Label encoding
label = LabelEncoder()
for i in train_housing.columns:
    if (train_housing[i].dtype == 'object'):
      train_housing[i] = label.fit_transform(train_housing[i].astype('str'))
      train_housing[i] = train_housing[i].astype('object')

for i in test_housing.columns:
    if (test_housing[i].dtype == 'object'):
      test_housing[i] = label.fit_transform(test_housing[i].astype('str'))
      test_housing[i] = test_housing[i].astype('object')

In [0]:
# Creating new variable 'origin'
train_housing['origin'] = 0
test_housing['origin'] = 1

In [0]:
test_housing.describe()

In [0]:
# Creating random samples from train-test data and combining them.
sample_train_housing = train_housing.sample(7662, random_state=12)
sample_test_housing = test_housing.sample(7000, random_state=11)  

sample_combined_housing = sample_train_housing.append(sample_test_housing)


In [0]:
sample_y_combined_housing = sample_combined_housing['origin'] # saving our 'origin' target variable.
sample_combined_housing = sample_combined_housing.drop('origin',axis=1) # dropping target variable from data for modelling.

In [0]:
# Creating model
model_sample_housing = RandomForestClassifier(n_estimators = 50, max_depth = 5,min_samples_leaf = 5)
drop_list_housing = []
for i in sample_combined_housing.columns:
  score = cross_val_score(model_sample_housing,pd.DataFrame(sample_combined_housing[i]),sample_y_combined_housing,cv=2,scoring='roc_auc')
  if (np.mean(score) > 0.8):
    drop_list_housing.append(i)
  print(i,np.mean(score))

In [0]:
# list of variables that have covariate shift
print(drop_list_housing)

Methods for dealing with covariate shift in housing market data

In [0]:
# One way to deal with covariate shift is to drop variables that have shift.
real_train_housing = train_housing.drop('origin',axis=1)
real_test_housing = test_housing.drop('origin',axis=1)

In [0]:
rf = RandomForestRegressor(n_estimators=200, max_depth=6,max_features=10)
rf.fit(real_train_housing.drop('price_doc',axis=1),real_train_housing['price_doc'])
pred = rf.predict(real_test_housing)

In [0]:
columns = ['price_doc']
submission = pd.DataFrame(data=pred,columns=columns)
submission['id'] = real_test_housing['id']
submission = submission[['id','price_doc']]
submission.to_csv('with_shift.csv', index=False)
# score in kaggle = 0.40207

In [0]:
### plotting importances
features = train_housing.drop('price_doc',axis=1).columns.values
importance = rf.feature_importances_
indices = np.argsort(importance)[::-1][:20]

#plot
plt.figure(figsize=(8,5))
plt.bar(range(len(indices)), importance[indices], color = 'b', align='center')
plt.xticks(range(len(indices)), features[indices], rotation='vertical')
plt.xlim([-1,len(indices)])
plt.show()

Now we'll drop less important variables and check if our score improves.

In [0]:
# After checking features with covariate shift and their importance we'll keep ‘life_sq’ and ‘kitch_sq’
# and drop the rest with covariate shift.

noshift_train_housing = real_train_housing.drop(['id','timestamp','hospital_beds_raion','cafe_sum_500_min_price_avg','cafe_sum_500_max_price_avg','cafe_avg_price_500'], axis=1)
noshift_test_housing = real_test_housing.drop(['id','timestamp','hospital_beds_raion','cafe_sum_500_min_price_avg','cafe_sum_500_max_price_avg','cafe_avg_price_500'], axis=1)

rf = RandomForestRegressor(n_estimators=200, max_depth=6,max_features=10)
rf.fit(noshift_train_housing.drop('price_doc',axis=1),noshift_train_housing['price_doc'])
noshift_pred = rf.predict(noshift_test_housing)
columns = ['price_doc']
sub = pd.DataFrame(data=noshift_pred,columns=columns)
sub['id'] = real_test_housing['id']
sub = sub[['id','price_doc']]
sub.to_csv('without_drifting.csv', index=False)

# score in kaggle = 0.40068